In [3]:
import numpy as np
import pandas as pd
import sklearn
import time
from data import *

In [4]:
global master1985, matchups
seedResultsCompact = createSeedResults1985DF(seeds, tourneyCompactResults)
master1985 = createSeedResultRegTotals1985DF(seedResultsCompact, regSeasCompactTotals)
matchups = [generateMatchups(1), generateMatchups(2), generateMatchups(3), generateMatchups(4)]

In [6]:
def tourneySim(year, customTeamsDF = None, detailed = True):
    #note that this implementation relies on matchups and the ref dataframe being global (i.e. accesible by the function without being explicitly passed)
    if detailed == True:
        masterTemp = master2003[:]
    elif detailed == False:
        masterTemp = master1985[:]

    masterTemp = masterTemp[masterTemp['Season' == 1985]]
    
    tourneyNet = []

    playInGames = masterTemp[(masterTemp['WNumSeed'] != round(masterTemp[masterTemp['WNumSeed']])) and (masterTemp['LNumSeed'] != round(masterTemp[masterTemp['LNumSeed']]))]

    for i in range(playInGames.shape[0]):
        

tourneySim(1997)

[[1, 16], [8, 9], [5, 12], [4, 13], [6, 11], [3, 14], [7, 10], [2, 15]]
  Season TeamID     G     W     L     Pts      PA
0   1985   1228  31.0  23.0   8.0  2115.0  1778.0
1   1985   1106  24.0  10.0  14.0  1719.0  1810.0
2   1985   1112  27.0  18.0   9.0  1796.0  1602.0
3   1985   1165  24.0  12.0  12.0  1473.0  1418.0
4   1985   1192  28.0  19.0   9.0  1901.0  1812.0


In [22]:
class tourneyGame:
    def __init__(self, evalFn, priorGameA = None, priorGameB = None, teamA = None, teamB = None):
        self.evalFn = evalFn
        self.priorGameA = priorGameA
        self.priorGameB = priorGameB
        self.teamA = teamA
        self.teamB = teamB

    def findWinner(self):
        if self.teamA == None:
            self.teamA = self.priorGameA.findWinner()
        if self.teamB == None:
            self.teamB = self.priorGameB.findWinner()
        
        self.winner = self.evalFn(self.teamA, self.teamB)

        return self.winner

In [19]:
def cock(a, b):
    return a + b

In [24]:
jack = []
jack.append(tourneyGame(evalFn = cock, teamA = 5, teamB = 7))
jack.append(tourneyGame(evalFn = cock, priorGameA = jack[0], teamB = 9))
jack.append(tourneyGame(evalFn = cock, teamA = 3, teamB = 4))
jack.append(tourneyGame(evalFn = cock, priorGameA = jack[1], priorGameB = jack[2]))

jack[3].findWinner()
jack[2].winner

7

In [8]:
seedResultsCompact = createSeedResults1985DF(seeds, tourneyCompactResults)
master1985 = createSeedResultRegTotals1985DF(seedResultsCompact, regSeasCompactTotals)

In [10]:
print('')
print(master1985[0:5])
print(master1985[master1985['Season'] == 1985].shape)
print(master1985.columns)


  Season  DayNum WTeamID  WScore  LTeamID  LScore WLoc  NumOT WSection  \
0   1985     136    1116      63     1234      54    N      0        X   
1   1985     136    1120      59     1345      58    N      0        Z   
2   1985     138    1120      66     1242      64    N      0        Z   
3   1985     136    1207      68     1250      43    N      0        W   
4   1985     138    1207      63     1396      46    N      0        W   

   WNumSeed  ...    WG  WWins  WLosses    WPts     WPA    LG  LWins  LLosses  \
0       9.0  ...  33.0   21.0     12.0  2156.0  2036.0  30.0   20.0     10.0   
1      11.0  ...  29.0   18.0     11.0  2040.0  1933.0  25.0   17.0      8.0   
2      11.0  ...  29.0   18.0     11.0  2040.0  1933.0  30.0   23.0      7.0   
3       1.0  ...  27.0   25.0      2.0  2045.0  1622.0  29.0   11.0     18.0   
4       1.0  ...  27.0   25.0      2.0  2045.0  1622.0  29.0   24.0      5.0   

     LPts     LPA  
0  2092.0  1778.0  
1  1728.0  1633.0  
2  2281.0  21

In [16]:
temp = ((1, 16), (8, 9), (5, 12), (4, 13), (6, 11), (3, 14), (7, 10), (2, 15))

temp2 = ((1, 8), (1, 9), (16, 8), (16, 9), (5, 4), (5, 13), (12, 4), (12, 13))

def generateGames(round):
    seedNums = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]

    out = []
    
    count = 0
    for i in range(2 ** round):
        groupSize = ((2 ** round) / 2)
        pivots = seedNums[count:(count + groupSize)]
        leaves = seedNums[(count + groupSize + 1):(count + groupSize + 1 + groupSize)]
        for j in pivots:
            for k in leaves:
                out.append([j, k])
    
    return out

generateGames(1)

TypeError: slice indices must be integers or None or have an __index__ method

In [14]:
jack = ((1, 2), (3, 4))
2 ** 3
jack[0:0]

()

In [ ]:
nrow = seedResults.shape[0]
seedResults['WExpected'] = seedResults.WTeamID

start = time.time()

for i in range(nrow):
    if seedResults.loc[i, 'WNumSeed'] >= seedResults.loc[i, 'LNumSeed']:
        seedResults.loc[i, 'WExpected'] = seedResults.loc[i, 'LTeamID']

end = time.time()

print(end - start)

seedResults['correctGuess'] = seedResults.WExpected == seedResults.WTeamID

In [ ]:
accuracy = seedResults.correctGuess.sum() / nrow
print("Picking highest seed strategy accuracy: ", accuracy)

In [6]:
seedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSection,WNumSeed,LSection,LNumSeed
0,1985,136,1116,63,1234,54,N,0,X,9.0,X,8.0
1,1985,136,1120,59,1345,58,N,0,Z,11.0,Z,6.0
2,1985,138,1120,66,1242,64,N,0,Z,11.0,Z,3.0
3,1985,136,1207,68,1250,43,N,0,W,1.0,W,16.0
4,1985,138,1207,63,1396,46,N,0,W,1.0,W,8.0


In [ ]:
def tourneyDriver(matchupsDF, referenceDF):
    matchups = matchupsDF[:]
    reference = referenceDF[:]